In [151]:
import tensorflow as tf
import keras
import numpy as np
import pandas as pd
from pca import *
from main import getLineidxs
from readply import expandIndices, getIndices, getNearestDist

In [152]:
def calcSpecificModel(eigenValues, mean, eigenVectors):
    U_k = eigenVectors[:, 0:30]
    print('eigenValues: ', eigenValues)
    eigenValues = eigenValues.transpose()
    x_redBase = copyList(eigenValues)
    x_red = copyList(eigenValues)
    print('eigenvalues shape:' , np.array(eigenValues).shape)
    basePCAmodel = (np.dot(U_k, x_redBase) + mean).real
    adjustedModel = (np.dot(U_k, x_red) + mean).real
    cutoff = 5
    indices = rp.getIndices('breast_indices/leftBreastIdxLargeArea.txt') + rp.getIndices('breast_indices/rightBreastIdxLargeArea.txt')
    outline = rp.getIndices('breast_indices/leftBreastIdxLargeAreaOutline.txt') + rp.getIndices('breast_indices/rightBreastIdxLargeAreaOutline.txt')
    indices = list(set(indices))
    dists = []
    for i in indices:
        dists.append(rp.getNearestDist(outline, adjustedModel, adjustedModel[i*3], adjustedModel[i*3+1], adjustedModel[i*3+2]))

    indices_exp = rp.expandIndices(indices)
    for i in range(len(indices_exp)):
        dist = dists[int(i/3)]
        if dist > cutoff:
            basePCAmodel[indices_exp[i]] = adjustedModel[indices_exp[i]]
        else:
            basePCAmodel[indices_exp[i]] = (dist/cutoff) * adjustedModel[indices_exp[i]] + (1-(dist/cutoff)) * basePCAmodel[indices_exp[i]]
    return basePCAmodel

def copyList(l):
    r = []
    for i in range(len(l)):
        r.append(l[i])
    return r

In [153]:
df_lines = pd.read_csv('lines.txt', sep=' ', header=None)
df_params = pd.read_csv('params.txt', sep=' ', header=None)
eigenVectors = read_matrix_from_hdf5('eigenVectors.h5')
mean = read_matrix_from_hdf5('mean.h5')
leftLineIdxs, rightLineIdxs = getLineidxs()

In [154]:
eigenValuesAugmentation_fit = [37.47394805, -49.07828283,  -0.6562043,  -24.62020633, -27.13628352, 6.57057748, -11.42703975, -18.79096601, -10.54424185, 3.44637848, -12.89579981, -7.87242574, -13.9092693, 2.27681137, -7.3060829, -7.73122333, -4.43121648, 5.80548907, 10.15422231, 14.24570948, -17.25150115, 0.97727416, -3.08962112, 5.8461985, 6.87403008, -3.17929581, 1.36895233, -0.74191196, -0.60536829, 5.94243889]
for i in range(30):
    df_lines[114+i] = [eigenValuesAugmentation_fit[i]] * len(df_lines)
train_data = df_lines.sample(frac=0.8,random_state=200)
test_data = df_lines.drop(train_data.index)
x_train = train_data
y_train = train_data.iloc[:, 0:114]
x_test = test_data
y_test = test_data.iloc[:, 0:114]

In [155]:
model = keras.Sequential([
    keras.layers.Dense(114, activation=tf.nn.relu, input_shape=(144,)),
    keras.layers.Dense(128, activation=tf.nn.relu),
    keras.layers.Dense(128, activation=tf.nn.relu),
    keras.layers.Dense(64, activation=tf.nn.relu),
    keras.layers.Dense(32, activation=tf.nn.relu),
    keras.layers.Dense(30, activation=tf.nn.relu),
])

class CustomLoss(keras.losses.Loss):
    def __init__(self):
        super().__init__()
        
    def call(self, y_true, y_pred):
        y_true = y_true.numpy()[0]
        y_pred = y_pred.numpy()[0]
        print('y_true: ', len(y_true))
        print('y_pred: ', len(y_pred))
        model = calcSpecificModel(y_pred, mean, eigenVectors)
        idxs = expandIndices(rightLineIdxs)
        sums = 0
        for i in range(0,len(idxs),3):
                sums += np.sqrt((model[idxs[i]] - y_true[i])**2 + (model[idxs[i+1]] - y_true[i+1])**2 + (model[idxs[i+2]] - y_true[i+2])**2)
        print('sums: ', sums)
        return tf.constant(sums, dtype=tf.float32)
    
loss_fn = CustomLoss()
model.compile(optimizer='adam', loss=loss_fn, run_eagerly=True)

In [156]:
model.fit(x_train, y_train, epochs=10, batch_size=1)

Epoch 1/10
y_true:  114
y_pred:  30
eigenValues:  [0.         0.         2.7692885  2.6379542  0.         0.
 5.7611547  0.         2.4482896  0.         2.7288687  2.6301765
 7.144993   2.163571   2.2425163  0.10416281 0.         1.8082294
 2.5232878  2.319271   0.         0.         3.0324333  0.
 0.43400693 0.         8.394964   0.         0.         1.9299822 ]
eigenvalues shape: (30,)
sums:  56.4375013253338


ValueError: No gradients provided for any variable: (['dense_150/kernel:0', 'dense_150/bias:0', 'dense_151/kernel:0', 'dense_151/bias:0', 'dense_152/kernel:0', 'dense_152/bias:0', 'dense_153/kernel:0', 'dense_153/bias:0', 'dense_154/kernel:0', 'dense_154/bias:0', 'dense_155/kernel:0', 'dense_155/bias:0'],). Provided `grads_and_vars` is ((None, <tf.Variable 'dense_150/kernel:0' shape=(144, 114) dtype=float32, numpy=
array([[-0.0491861 , -0.09003512,  0.02177499, ..., -0.00919415,
        -0.0322129 , -0.1002748 ],
       [-0.08161794, -0.09800388,  0.10288084, ...,  0.13848191,
         0.02446996,  0.08145641],
       [-0.04487093, -0.02741075,  0.05417484, ...,  0.08360934,
        -0.11829676,  0.01522577],
       ...,
       [-0.08199291,  0.0572775 , -0.12745407, ..., -0.0317951 ,
         0.02575523,  0.02260385],
       [ 0.08542813,  0.0931437 , -0.05722107, ...,  0.04870391,
        -0.09060395,  0.1095778 ],
       [-0.0451086 , -0.02803287,  0.12121722, ...,  0.00358029,
    